In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# tullock alpha < 1

In [151]:
b1, b2, v1, v2 = sp.symbols('b_1 b_2 v_1 v_2')

a = 1/2

x1 = b1**a / (b1**a + b2**a)
x2 = b2**a / (b1**a + b2**a)

u1 = v1 * x1 - b1
u2 = v2 * x2 - b2
u2

b_2**0.5*v_2/(b_1**0.5 + b_2**0.5) - b_2

In [152]:
part1 = sp.simplify(sp.diff(u1, b1))
part2 = sp.simplify(sp.diff(u2, b2))
part2

0.5*b_1**0.5*v_2/(b_2**0.5*(b_1**0.5 + b_2**0.5)**2) - 1

In [166]:
func1 = sp.lambdify([b1, b2], part1.evalf(subs={v1:200.}))
func2 = sp.lambdify([b1, b2], part2.evalf(subs={v2:1.}))

In [167]:
def eqs(p):
    b1, b2 = p
    return [func1(b1, b2), func2(b1, b2)]

sol = fsolve(eqs, (0.49, 0.12))
sol, sol[0]/sol[1]

<lambdifygenerated-79>:2: RuntimeWarning: invalid value encountered in sqrt
  return 100.0*b_1**(-0.5)*sqrt(b_2)/(sqrt(b_1) + sqrt(b_2))**2 - 1.0
<lambdifygenerated-80>:2: RuntimeWarning: invalid value encountered in double_scalars
  return 0.5*sqrt(b_1)*b_2**(-0.5)/(sqrt(b_1) + sqrt(b_2))**2 - 1.0
<lambdifygenerated-80>:2: RuntimeWarning: invalid value encountered in sqrt
  return 0.5*sqrt(b_1)*b_2**(-0.5)/(sqrt(b_1) + sqrt(b_2))**2 - 1.0


(array([3.13932985, 0.03938495]), 79.70886000804566)